# Final Model Analysis
In this notebook, we perform analysis on the output of the final model. In particular, we analyse the model predictions of the official dev set.

In [24]:
import pandas as pd
from sklearn.metrics import f1_score

with open("data/dev.txt", "r") as file:
    predictions = [int(line.strip()) for line in file]

df = pd.read_csv("data/dev.csv")
df["prediction"] = predictions

f1_score(df["label"], df["prediction"], pos_label=1)

0.6101694915254238